In [1]:
import os
import torch
import numpy as np
from sklearn import preprocessing

In [2]:
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## Loading Dataset

In [4]:
class Custom_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False, train=False):
        self.train = train
        self.loadedFrom = file
        
        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.X, self.Y = data[:,0:n_col-1].astype(np.float64), data[:,n_col-1].astype(np.float64)
    
        # oe = preprocessing.OrdinalEncoder()
        # oe.fit(self.X)
        # self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(np.array(self.Y[idx]))

In [5]:
train_path = './data/banknote/train.csv'
test_path = './data/banknote/test.csv'

In [6]:
dataset_train = Custom_Dataset(train_path, train=True)

In [7]:
train_batch_size = 1
train_size = len(dataset_train)

In [8]:
train_loader = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=True)

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_shape=14):
        super(NeuralNetwork, self).__init__()        
        self.stack = nn.Sequential(
            nn.Linear(input_shape, 10),
            nn.Sigmoid(),
            nn.Linear(10, 10),
            nn.Sigmoid(),
            nn.Linear(10, 2),
        )
        
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(module)
                nn.init.uniform_(module.weight)
        
        self.loss = nn.L1Loss()
        self.double()
        
    def forward(self, x):        
        h = x
        for layer in self.stack:
            h = layer(h)
        return h

In [10]:
model = NeuralNetwork(input_shape=4).to(device)
print(model)

Linear(in_features=4, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=2, bias=True)
NeuralNetwork(
  (stack): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=10, out_features=2, bias=True)
  )
  (loss): L1Loss()
)


In [11]:
learning_rate = 1

print("Starting Iteration")
for t in range(20):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    for i in range(train_size//train_batch_size):
        x_tr, y_tr = next(iter(train_loader))
        y_pred = model(x_tr)

        # Compute and print loss. We pass Tensors containing the predicted and true
        # values of y, and the loss function returns a Tensor containing the
        # loss.
        loss = model.loss(y_pred, y_tr)

        # Zero the gradients before running the backward pass.
        model.zero_grad()

        # Backward pass: compute gradient of the loss with respect to all the learnable
        # parameters of the model. Internally, the parameters of each Module are stored
        # in Tensors with requires_grad=True, so this call will compute gradients for
        # all learnable parameters in the model.
        loss.backward()

        # Update the weights using gradient descent. Each parameter is a Tensor, so
        # we can access its gradients like we did before.
        with torch.no_grad():
            learning_rate = learning_rate/1+t
            for param in model.parameters():
                param -= learning_rate * param.grad
    
    with torch.no_grad():
        random_ids = np.random.randint(train_size, size=train_batch_size)
        random_x, random_y = dataset_train[random_ids]
        pred = torch.flatten(model(random_x))
        print("pred_", pred, "target", random_y)
        pred = np.argmax(pred.numpy())
        print("pred", pred, "target", random_y)
        l = model.loss(torch.Tensor(pred), random_y).to(torch.double)
        print("completing " + str(t) + " rounds, loss: " + str(l))

Starting Iteration


/Users/albertliu/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


pred_ tensor([1.1848, 0.6991], dtype=torch.float64) target tensor([1.], dtype=torch.float64)
pred 0 target tensor([1.], dtype=torch.float64)
completing 0 rounds, loss: tensor(nan, dtype=torch.float64)


/Users/albertliu/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([0])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


pred_ tensor([-218.1355,  218.2623], dtype=torch.float64) target tensor([0.], dtype=torch.float64)
pred 1 target tensor([0.], dtype=torch.float64)
completing 1 rounds, loss: tensor(0., dtype=torch.float64)
pred_ tensor([ 653.8646, -653.7376], dtype=torch.float64) target tensor([1.], dtype=torch.float64)
pred 0 target tensor([1.], dtype=torch.float64)
completing 2 rounds, loss: tensor(nan, dtype=torch.float64)
pred_ tensor([-1307.1355,  1307.2623], dtype=torch.float64) target tensor([0.], dtype=torch.float64)
pred 1 target tensor([0.], dtype=torch.float64)
completing 3 rounds, loss: tensor(0., dtype=torch.float64)
pred_ tensor([ 2178.3645, -2178.2377], dtype=torch.float64) target tensor([0.], dtype=torch.float64)
pred 0 target tensor([0.], dtype=torch.float64)
completing 4 rounds, loss: tensor(nan, dtype=torch.float64)
pred_ tensor([-3267.1355,  3267.2623], dtype=torch.float64) target tensor([0.], dtype=torch.float64)
pred 1 target tensor([0.], dtype=torch.float64)
completing 5 rounds, 

In [12]:
'''
class Test_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False):
        self.loadedFrom = file

        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.id, self.X = data[:,0:1], data[:,1:n_col]
    
        oe = preprocessing.OrdinalEncoder()
        oe.fit(self.X)
        self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx]
'''

class Test_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False):
        self.loadedFrom = file

        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.X, self.Y = data[:,0:n_col-1].astype(np.float64), data[:,n_col-1].astype(np.float64)
    
        # oe = preprocessing.OrdinalEncoder()
        # oe.fit(self.X)
        # self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [13]:
dataset_test = Test_Dataset(test_path, encode_categorical=True)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

In [17]:
# Disable grad
with torch.no_grad():
    
    for i in range(100):
        _x, _y = np.asarray(dataset_test[i])
        item = torch.from_numpy(_x)
        
        # Generate prediction
        print("_x=", item)
        prediction = model(item)
        # Predicted class value using argmax
        predicted_class = np.argmin(prediction)
        print("prediction=", prediction, "i=", predicted_class)

_x= tensor([ 2.8521,  9.1710, -3.6461, -1.2047], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([ 5.2418, 10.5388, -4.1174, -4.2797], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([-2.2623, 12.1177,  0.2885, -7.7581], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([ 0.5530, -3.4619,  1.7048,  1.1008], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([ 4.1542,  7.2756, -2.4766, -1.2099], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([-1.7279, -6.8410,  8.9494,  0.6806], dtype=torch.float64)
prediction= tensor([-41375.1355,  41375.2623], dtype=torch.float64) i= tensor(0)
_x= tensor([-1.4454, -8.4385,  8.8483,  0.9689], dtype=torch.float64)
prediction= tensor([-413

/var/folders/sf/c7hgl8d17_n88p784_hhy2lh0000gn/T/ipykernel_4353/1432655199.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  _x, _y = np.asarray(dataset_test[i])
